## Load Data

In [1]:
from src.morse import Morse
from src.evaluation_and_conversion import compare_result_txt_to_groundtruth_ply
import itertools

In [2]:
def process(artifact, persistence, params, index = 7):
    data_conf = Morse()
    data_n = Morse()
    data_conf.load_mesh_ply("Data/"+artifact+".ply", quality_index=index, inverted=True)
    data_n.load_mesh_ply("Data/"+artifact+".ply", quality_index=index, inverted=True)

    outfilename =  "Data/results/" + artifact + "/" + artifact

    #### CONFORMING
    data_conf.load_labels("Data/"+artifact+"_labels.txt")
    data_conf.ConformingGradient()
    data_conf.ExtractMorseComplex()

    accuracy = 0
    best_params = []

    for pers in persistence:
        data_conf.ReduceMorseComplex(pers, conforming=True)
        data_conf.ExtractMorseCells(pers)

        for high, low, merge in params:
            if high > low:
                data_conf.Segmentation(pers, high, low, merge, conforming=True)
                data_conf.plot_Segmentation_label_txt(pers, high, low, merge, outfilename)
                result = compare_result_txt_to_groundtruth_ply(outfilename+ "_" + str(pers) + "P_" + str(high) + "-" + str(low) 
                                       + "T_" + str(merge) + ".txt", "Data/"+artifact+"_c.ply")
                if result[0] > accuracy:
                    accuracy = result[0]
                    best_params = result[1:]

    print(accuracy, best_params)

    ### NON CONFORMING
    data_n.ProcessLowerStars()
    data_n.ExtractMorseComplex()

    data_n.ReduceMorseComplex(best_params[0], conforming=False)
    data_n.ExtractMorseCells(best_params[0])
    data_n.Segmentation(best_params[0], best_params[1], best_params[2], best_params[3], conforming=False)
    data_n.plot_Segmentation_label_txt(best_params[0], best_params[1], best_params[2], best_params[3], outfilename+"non_conf")




In [3]:
# PARAMETERS
persistence = [0.03, 0.04, 0.05]
high_thresh = [0.06, 0.07, 0.08]
low_thresh = [0.02, 0.03, 0.04, 0.05]
merge_thresh = [0.3, 0.4, 0.5]


process("700", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)))

load_mesh_ply took 19.20294 seconds to execute!
load_mesh_ply took 28.27909 seconds to execute!
Time read labels in txt file: 0.15208359993994236
Time ProcessLowerStar: 6.893067399971187
ExtractMorseComplex took 5.60337 seconds to execute!
ReduceMorseComplex took 5.76668 seconds to execute!
BD points:  41688
ExtractMorseCells took 2.95197 seconds to execute!
Need maximally reduced complex for salient edges...
Persistence was high enough that this complex is maximally reduced.
ReduceMorseComplex took 10.49499 seconds to execute!
get_salient_edges took 0.11196 seconds to execute!
Segmentation took 30.00761 seconds to execute!
plot_Segmentation_label_txt took 0.44909 seconds to execute!
compare_result_txt_to_groundtruth_ply took 12.85053 seconds to execute!
get_salient_edges took 0.12639 seconds to execute!
Segmentation took 20.41587 seconds to execute!
plot_Segmentation_label_txt took 0.27475 seconds to execute!
compare_result_txt_to_groundtruth_ply took 10.53014 seconds to execute!
get_

In [4]:
# PARAMETERS
persistence = [0.03, 0.04, 0.05]
high_thresh = [0.04, 0.05, 0.06]
low_thresh = [0.02, 0.03, 0.04]
merge_thresh = [0.1, 0.2, 0.3]

process("801", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)), index = 3)

load_mesh_ply took 21.44164 seconds to execute!
load_mesh_ply took 20.88740 seconds to execute!
Time read labels in txt file: 0.11172439996153116
Time ProcessLowerStar: 6.2518251000437886
ExtractMorseComplex took 6.34803 seconds to execute!
ReduceMorseComplex took 4.09324 seconds to execute!
BD points:  53106
ExtractMorseCells took 2.00489 seconds to execute!
Need maximally reduced complex for salient edges...
Persistence was high enough that this complex is maximally reduced.
ReduceMorseComplex took 4.40124 seconds to execute!
get_salient_edges took 0.08078 seconds to execute!
Segmentation took 15.56103 seconds to execute!
plot_Segmentation_label_txt took 0.22241 seconds to execute!
compare_result_txt_to_groundtruth_ply took 9.57789 seconds to execute!
get_salient_edges took 0.09275 seconds to execute!
Segmentation took 14.23113 seconds to execute!
plot_Segmentation_label_txt took 0.23138 seconds to execute!
compare_result_txt_to_groundtruth_ply took 9.55283 seconds to execute!
get_sa

In [5]:
# PARAMETERS
persistence = [0.06, 0.07, 0.08]
high_thresh = [0.08, 0.09, 0.1]
low_thresh = [0.06, 0.07, 0.08]
merge_thresh = [0.2, 0.3, 0.4]

process("4501", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)))

load_mesh_ply took 17.26305 seconds to execute!
load_mesh_ply took 19.93653 seconds to execute!
Time read labels in txt file: 0.12839339999482036
Time ProcessLowerStar: 7.467729300027713
ExtractMorseComplex took 4.94065 seconds to execute!
ReduceMorseComplex took 9.25874 seconds to execute!
BD points:  43069
ExtractMorseCells took 4.53274 seconds to execute!
Need maximally reduced complex for salient edges...
Persistence was high enough that this complex is maximally reduced.
ReduceMorseComplex took 6.37387 seconds to execute!
get_salient_edges took 0.04687 seconds to execute!
Segmentation took 25.34427 seconds to execute!
plot_Segmentation_label_txt took 0.24138 seconds to execute!
compare_result_txt_to_groundtruth_ply took 10.88471 seconds to execute!
get_salient_edges took 0.04987 seconds to execute!
Segmentation took 20.42887 seconds to execute!
plot_Segmentation_label_txt took 0.24498 seconds to execute!
compare_result_txt_to_groundtruth_ply took 10.42162 seconds to execute!
get_s

In [6]:
print('700:')
result = compare_result_txt_to_groundtruth_ply("Data/results/700/700_0.03P_0.06-0.02T_0.4.txt", "Data/700_c.ply")  
print(result[0])   
result = compare_result_txt_to_groundtruth_ply("Data/results/700/700non_conf_0.03P_0.06-0.02T_0.4.txt", "Data/700_c.ply")  
print(result[0])  

print('801:')
result = compare_result_txt_to_groundtruth_ply("Data/results/801/801_0.04P_0.06-0.04T_0.2.txt", "Data/801_c.ply")     
print(result[0])  
result = compare_result_txt_to_groundtruth_ply("Data/results/801/801non_conf_0.04P_0.06-0.04T_0.2.txt", "Data/801_c.ply")          
print(result[0])  

print('4501:')
result = compare_result_txt_to_groundtruth_ply("Data/results/4501/4501_0.07P_0.1-0.07T_0.3.txt", "Data/4501_c.ply")     
print(result[0])  
result = compare_result_txt_to_groundtruth_ply("Data/results/4501/4501non_conf_0.07P_0.1-0.07T_0.3.txt", "Data/4501_c.ply")    
print(result[0])   

700:
compare_result_txt_to_groundtruth_ply took 9.29391 seconds to execute!
92.49944870998135
compare_result_txt_to_groundtruth_ply took 12.57962 seconds to execute!
73.77363029489004
801:
compare_result_txt_to_groundtruth_ply took 9.04221 seconds to execute!
94.85600282446299
compare_result_txt_to_groundtruth_ply took 9.05347 seconds to execute!
79.80845209340902
4501:
compare_result_txt_to_groundtruth_ply took 10.38498 seconds to execute!
98.02251977480225
compare_result_txt_to_groundtruth_ply took 10.07311 seconds to execute!
97.72552274477255


## Results

### 700
Persistence: 0.03

Thresh_high: 0.06

Thresh_low: 0.02

Thresh_merge: 0.4


Accuracy conforming: 92.5 %

Accuracy non conforming: 73.77 %

### 801
Persistence: 0.04

Thresh_high: 0.06

Thresh_low: 0.04

Thresh_merge: 0.2


Accuracy conforming: 94.86 %

Accuracy non conforming: 79.81 %

### 4501
Persistence: 0.07

Thresh_high: 0.1

Thresh_low: 0.07

Thresh_merge: 0.3


Accuracy conforming: 98.02 %

Accuracy non conforming: 97.73 %